In [ ]:
import pandas as pd

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('/content/drive/MyDrive/DNA/dive 2024/dive2024데이터프레임 - 최종df-2.csv')

In [ ]:
# prompt: minmax

from sklearn.preprocessing import MinMaxScaler

columns_to_scale = df.drop(['법정동','인구비율_65세이상', 'Latitude', 'Longitude','고령부상자수'], axis=1).columns

scaler = MinMaxScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

# 다중공선성

In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

#X_train should be a DataFrame, not an Index object
X_train = df.drop(['법정동','고령운전자부상비중', '인구비율_65세이상', 'Latitude', 'Longitude'], axis=1)

def feature_engineering_XbyVIF(X_train):
    vif = pd.DataFrame()
    vif['VIF_Factor'] = [variance_inflation_factor(X_train.values, i)
                         for i in range(X_train.shape[1])]
    vif['Feature'] = X_train.columns
    return vif
vif = feature_engineering_XbyVIF(X_train)
print(vif)

    VIF_Factor      Feature
0     5.341984     어린이보호구역수
1     1.498664        노면문자수
2     5.616182         승용차수
3     3.663709     교통신호제어기수
4     1.198886         교차로수
5     5.530020         중학교수
6     3.193286        안전지대수
7     4.622762     동물포획출동횟수
8    12.190957       버스정류장수
9     2.093360   노인장애인보호구역수
10    2.672097       과속방지턱수
11    5.052792    이륜차사고부상자수
12   10.913884    보행자사고부상자수
13    4.874634  고령보행자사고부상자수
14    1.698640     음주사고부상자수
15    1.261347    스쿨존사고부상자수
16    1.293271  보행어린이사고부상자수
17    2.858963   무단횡단사고부상자수
18    1.056979    자전거사고부상자수


# 회귀

In [ ]:
!pip install libpysal

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 20.6 MB/s eta 0:00:00


In [ ]:
!pip install esda

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.5 MB/s eta 0:00:00


In [ ]:
!pip install splot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.8/372.8 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 16.1 MB/s eta 0:00:00


In [ ]:
!pip install stargazer

In [ ]:
!pip install mgwr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.9 MB/s eta 0:00:00


In [ ]:
# Load libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from pylab import rcParams
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 72

import seaborn as sns
sns.set_style("darkgrid")
sns.set_context(context="paper", font_scale=1.5, rc=None)
sns.set(font="serif")

import plotly.express as px
import plotly.graph_objects as go

import geopandas as gpd


import libpysal as ps
from libpysal  import weights
from libpysal.weights import Queen

import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster, plot_local_autocorrelation
from splot.libpysal import plot_spatial_weights

from giddy.directional import Rose

import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer, LineLocation

from spreg import OLS
from spreg import MoranRes
from spreg import ML_Lag
from spreg import ML_Error

from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap

import warnings
warnings.filterwarnings('ignore')
import time

In [ ]:
df.columns

Index(['법정동', 'Latitude', 'Longitude', '고령운전자부상비중', '어린이보호구역수', '노면문자수',
       '인구비율_65세이상', '승용차수', '교통신호제어기수', '교차로수', '중학교수', '안전지대수', '동물포획출동횟수',
       '버스정류장수', '노인장애인보호구역수', '과속방지턱수', '이륜차사고부상자수', '보행자사고부상자수',
       '고령보행자사고부상자수', '음주사고부상자수', '스쿨존사고부상자수', '보행어린이사고부상자수', '무단횡단사고부상자수',
       '자전거사고부상자수', '고령부상자수'],
      dtype='object')

교통신호제어기수 0.9
과속 방지턱수 0.9

In [ ]:
X = df[['어린이보호구역수', '동물포획출동횟수', '승용차수', '노인장애인보호구역수', '이륜차사고부상자수','고령보행자사고부상자수',
        '음주사고부상자수', '스쿨존사고부상자수', '보행어린이사고부상자수','자전거사고부상자수']].values
X.shape

(191, 10)

In [ ]:
y = df['고령부상자수'].values.reshape((-1,1)) # reshape is needed to have column array
y.shape

(191, 1)

In [ ]:
u = df['Latitude']
v = df['Longitude']
coords = list(zip(u,v))

In [ ]:
%%time
gwr_selector = Sel_BW(coords, y, X)
gwr_bw = gwr_selector.search()

LinAlgError: Matrix is singular.

In [ ]:
print('GWR bandwidth =', gwr_bw)

NameError: name 'gwr_bw' is not defined

In [ ]:
gwr_results = GWR(coords, y, X, gwr_bw).fit()
gwr_results.summary()

In [ ]:
# As reference, here is the (average) R2, AIC, and AICc
print('Mean R2 =', gwr_results.R2)
print('AIC =', gwr_results.aic)
print('AICc =', gwr_results.aicc)

# 모델링(회귀)

In [ ]:
!pip install libpysal
!pip install geopandas
!pip install esda
!pip install splot
!pip install mgwr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.8/372.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.4/215.4 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.6 MB/s eta 0:00:00


In [ ]:
!pip install stargazer

In [ ]:
# Load libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from pylab import rcParams
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 72

import seaborn as sns
sns.set_style("darkgrid")
sns.set_context(context="paper", font_scale=1.5, rc=None)
sns.set(font="serif")

import plotly.express as px
import plotly.graph_objects as go

import geopandas as gpd


import libpysal as ps
from libpysal  import weights
from libpysal.weights import Queen

import esda
from esda.moran import Moran, Moran_Local

import splot
from splot.esda import moran_scatterplot, plot_moran, lisa_cluster, plot_local_autocorrelation
from splot.libpysal import plot_spatial_weights

from giddy.directional import Rose

import statsmodels.api as sm
import statsmodels.formula.api as smf
from stargazer.stargazer import Stargazer, LineLocation

from spreg import OLS
from spreg import MoranRes
from spreg import ML_Lag
from spreg import ML_Error

from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap, truncate_colormap

import warnings
warnings.filterwarnings('ignore')
import time

In [ ]:
from shapely.geometry import Point # import the Point object from shapely.geometry

In [ ]:
final_df['geometry'] = final_df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

# GeoDataFrame으로 변환
gdf = gpd.GeoDataFrame(final_df, geometry='geometry', crs="EPSG:4326")

In [ ]:
X = gdf[['승용차수',  '안전지대수', '동물포획출동횟수',
       '노인장애인보호구역수', '이륜차사고부상자수', '고령보행자사고부상자수', '음주사고부상자수',
        '보행어린이사고부상자수', '자전거사고부상자수']].values
X.shape


(191, 9)

In [ ]:
y = gdf['고령부상자수'].values.reshape((-1,1)) # reshape is needed to have column array
y.shape

(191, 1)

In [ ]:
u = gdf['Latitude']
v = gdf['Longitude']
coords = list(zip(u,v))

In [ ]:
%%time
gwr_selector = Sel_BW(coords, y, X)
gwr_bw = gwr_selector.search()

CPU times: user 259 ms, sys: 26.7 ms, total: 286 ms
Wall time: 546 ms


In [ ]:
print('GWR bandwidth =', gwr_bw)

GWR bandwidth = 110.0


In [ ]:
gwr_results = GWR(coords, y, X, gwr_bw).fit()
gwr_results.summary()

Model type                                                         Gaussian
Number of observations:                                                 191
Number of covariates:                                                    10

Global Regression Results
---------------------------------------------------------------------------
Residual sum of squares:                                          17892.128
Log-likelihood:                                                    -704.572
AIC:                                                               1429.144
AICc:                                                              1432.619
BIC:                                                              16941.467
R2:                                                                   0.812
Adj. R2:                                                              0.803

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [ ]:
# As reference, here is the (average) R2, AIC, and AICc
print('Mean R2 =', gwr_results.R2)
print('AIC =', gwr_results.aic)
print('AICc =', gwr_results.aicc)

Mean R2 = 0.8985643360351913
AIC = 1366.3605670644745
AICc = 1385.3175821088805


In [ ]:
final_df.to_csv("모델링데이터.csv", index=False)

In [ ]:
gdf['gwr_intercept'] = gwr_results.params[:,0]
gdf['gwr_fb']        = gwr_results.params[:,1]
gdf['gwr_aa']        = gwr_results.params[:,2]
gdf['gwr_rural']     = gwr_results.params[:,3]

In [ ]:
gdf['gwr_rural']

,gwr_rural
0,10.361454
1,10.837118
2,10.856959
3,10.287039
4,10.373423
...,...
186,4.822978
187,2.859152
188,3.225979
189,3.471647
